In [1]:
from snowflake.ml.feature_store import CreationMode, FeatureStore, FeatureView, Entity

from snowflake.snowpark import Window
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *


session = get_active_session()

fs = FeatureStore(
    session=session, 
    database=session.get_current_database(),
    name=session.get_current_schema(),
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


## Feature View Computation & Registration: Airport Weather

In [3]:
raw_df = session.table("AIRPORT_WEATHER_STATION")
raw_df.to_pandas().head(5)

,DATETIME_UTC,AIRPORT_ZIP_CODE,RAIN_MM_H
0,2024-03-11 02:40:00,33126,0.0
1,2024-03-11 02:41:00,33126,0.0
2,2024-03-11 02:42:00,33126,0.0
3,2024-03-11 02:43:00,33126,0.0
4,2024-03-11 02:44:00,33126,0.0


In [4]:
window = Window.partition_by("AIRPORT_ZIP_CODE").order_by("DATETIME_UTC").rows_between(-29, Window.CURRENT_ROW)
window2 = Window.partition_by("AIRPORT_ZIP_CODE").order_by("DATETIME_UTC").rows_between(-59, Window.CURRENT_ROW)
transformed_df = raw_df.select(
    "DATETIME_UTC",
    "AIRPORT_ZIP_CODE",
    avg("RAIN_MM_H").over(window).alias("AVG30MIN_RAIN_MM_H"),
    avg("RAIN_MM_H").over(window2).alias("AVG60MIN_RAIN_MM_H")
)

transformed_df.limit(5).to_pandas()

0.4130330538672536

In [ ]:
raw_df.analytics.moving_agg(
    aggs={"RAIN_MM_H": ["AVG"]},
    window_sizes=[30, 60],
    group_by=["AIRPORT_ZIP_CODE"],
    order_by=["AIRPORT_ZIP_CODE", "DATETIME_UTC"],
).select(
    "DATETIME_UTC",
    "AIRPORT_ZIP_CODE",
    "RAIN_MM_H_AVG_30",
    "RAIN_MM_H_AVG_60"
).to_pandas().head()

In [6]:
airport_entity = Entity(name="AIRPORT_ZIP_CODE", join_keys=["AIRPORT_ZIP_CODE"], desc='An Entity Description')
fs.register_entity(airport_entity)
fs.list_entities().show()

--------------------------------------------------------------
|"NAME"            |"JOIN_KEYS"           |"DESC"  |"OWNER"  |
--------------------------------------------------------------
|AIRPORT_ZIP_CODE  |["AIRPORT_ZIP_CODE"]  |        |MILES    |
--------------------------------------------------------------



In [7]:
fv = FeatureView(
    name="WEATHER_FEATURES",
    entities = [airport_entity],
    feature_df=transformed_df,
    timestamp_col = 'DATETIME_UTC',
    refresh_freq="1 minutes",
    desc='A FeatureView'
)

In [8]:
registered_fv = fs.register_feature_view(
    feature_view=fv,
    version="V1",
    overwrite=True
)

## Feature View Computation & Registration: Plane Attributes

In [9]:
plane_df = session.table('PLANE_MODEL_ATTRIBUTES').select(col('PLANE_MODEL'), col('SEATING_CAPACITY'))
plane_df.to_pandas()

,PLANE_MODEL,SEATING_CAPACITY
0,Boeing 737,150
1,Boeing 737 MAX,180
2,Boeing 747,400
3,Boeing 767,290
4,Boeing 777,368
5,Boeing 787 Dreamliner,335
6,Airbus A320,150
7,Airbus A320neo,165
8,Airbus A330,277
9,Airbus A340,375


In [10]:
plane_entity = Entity(name="PLANE_MODEL", join_keys=["PLANE_MODEL"])
fs.register_entity(plane_entity)
fs.list_entities().show()

--------------------------------------------------------------
|"NAME"            |"JOIN_KEYS"           |"DESC"  |"OWNER"  |
--------------------------------------------------------------
|AIRPORT_ZIP_CODE  |["AIRPORT_ZIP_CODE"]  |        |MILES    |
|PLANE_MODEL       |["PLANE_MODEL"]       |        |MILES    |
--------------------------------------------------------------



In [11]:
fv = FeatureView(
    name="PLANE_FEATURES",
    entities = [plane_entity],
    feature_df=plane_df,
    #no refresh--views
)

registered_fv = fs.register_feature_view(
    feature_view=fv,
    version="V1",
    overwrite=True
)